Оглавление:
1. [Подготовка датасетов](#Подготовка)
2. [Затраты на каждый рекламный источник](#Затраты)
3. [Суммарная выручка, максимальная и минимальнаяю выручку от заказов, устройство desktop, самый дорогой рекламный источник](#Выручка)
4. [Вычисление метрики](#Метрика)

#### 1. Подготовка датасетов <a id="Подготовка"></a>

In [13]:
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'analyst',         # имя пользователя
             'pwd': 'I80qcMa', # пароль
             'host': 'logdb.cdxwh80nuzgz.us-west-2.rds.amazonaws.com',
             'port': 5432,              # порт подключения
             'db': 'shopinfo'}  
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                        db_config['pwd'],
                                                        db_config['host'],
                                                        db_config['port'],
                                                        db_config['db'])

engine = create_engine(connection_string)
query_v = '''SELECT * FROM visits'''
query_s = '''SELECT * FROM orders'''
query_a = '''SELECT * FROM advertisment_costs'''
visits = pd.io.sql.read_sql(query_v, con=engine)
orders = pd.io.sql.read_sql(query_s, con=engine)
advertisment_costs = pd.io.sql.read_sql(query_a, con=engine)  

#### 2. Посчитайте затраты на каждый рекламный источник <a id="Затраты"></a>

##### SQL

In [2]:
sum_ad = '''
SELECT
    source_id,
    SUM(costs)
FROM 
    advertisment_costs 
GROUP BY
    source_id
ORDER BY
    source_id ASC
'''
sum_ad = pd.io.sql.read_sql(sum_ad, con=engine)
sum_ad

,source_id,sum
0,1,"$20,833.27"
1,2,"$42,806.04"
2,3,"$141,321.63"
3,4,"$61,073.60"
4,5,"$51,757.10"
5,9,"$5,517.49"
6,10,"$5,822.49"


#### Python

In [26]:
advertisment_costs['costs'] = advertisment_costs['costs'].map(lambda x: x.replace('$','')
                                                                  .replace(',','')).astype('float')
advertisment_costs.groupby('source_id', as_index=False)['costs'].sum()

,source_id,costs
0,1,20833.27
1,2,42806.04
2,3,141321.63
3,4,61073.60
4,5,51757.10
5,9,5517.49
6,10,5822.49


##### Вывод
Выстроены точно по возрастанию затрат. Временной промежуток с 01.06.2017 по 31.05.2018. Количество уникальных дат по затратам на рекламный источник у всех одинаково (363), кроме источника 5. У него на 1 дату больше. А именно на 31.03.2018.

------------

# Комментарии 1

Отлично :)

------------

####  <a id="Выручка"></a> 3. Посчитайте суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК (тип устройства desktop) пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег. Сделайте выводы

##### SQL

In [17]:
sum_rev = '''
SELECT 
    MAX(revenue),
    MIN(revenue),
    SUM(revenue)
FROM 
    (SELECT uid, device, sourceid FROM visits WHERE device = 'desktop' AND sourceid=3) AS ac
INNER JOIN
    orders ON orders.uid = ac.uid
'''
sum_rev = pd.io.sql.read_sql(sum_rev, con=engine)
sum_rev

,max,min,sum
0,$229.78,$0.00,"$18,879.56"


In [18]:
count_devices = '''
SELECT 
    device,
    COUNT(device) 
FROM 
    visits 
WHERE 
    sourceid = 3 
GROUP BY 
    device
'''
count_devices = pd.io.sql.read_sql(count_devices,con=engine)
count_devices

,device,count
0,desktop,5159
1,touch,1480


#### Python

In [14]:
revenue_3 = visits[(visits['sourceid'] == 3) & (visits['device'] == 'desktop')].reset_index()
visits_uid = revenue_3['uid'].unique()
revenue_3_desktop = orders.query('uid in @visits_uid')
revenue_3_desktop['revenue'] = revenue_3_desktop['revenue'].map(lambda x: x.replace('$','')
                                                                  .replace(',','.')).astype('float')

print('Максимальная выручка с устройста desktop', revenue_3_desktop['revenue'].max())
print('Минимальная выручка с устройста desktop', revenue_3_desktop['revenue'].min())
print('Суммарная выручка с устройста desktop', revenue_3_desktop['revenue'].sum())

Максимальная выручка с устройста desktop 229.78
Минимальная выручка с устройста desktop 0.0
Суммарная выручка с устройста desktop 13330.09


C:\Users\Arina\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [15]:
ty = visits[visits['sourceid'] == 3]
ty.groupby('device')['uid'].count()

device
desktop    5159
touch      1480
Name: uid, dtype: int64

#### Вывод
Получается, что доходы только с ПК минимальны и не покрывают расходы на самый дорогой рекламный источник. Учитывая, что переходы из данного рекламного источника в 88% случаев происходят из устройств desktop (5159) и touch (1480), результаты неутешительные. Доходы с touch не покрывают эту пропасть. Необходимо пересмотреть данный вид расхода средств.

#### <a id="Метрика"></a> Ведём новую метрику эффективности рекламы — дневные затраты на рекламу на пользователя по источнику. Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника.

#### 4. Рассчитайте для каждого дня введённую метрику. Считайте, что целевое действие было произведено тогда, когда сессия началась. Например, если посетитель провёл на сайте несколько минут с 23:50 14 апреля по 0:20 15 апреля, то его нужно включать в расчёт за 14 апреля 

##### SQL

In [53]:
metric = '''
WITH spend_ad AS (SELECT
                    source_id,
                    DATE_TRUNC('day',dt::TIMESTAMP) AS dt_day,
                    SUM(costs)                    
                FROM
                    advertisment_costs
                GROUP BY
                    source_id,
                    dt_day
                    ),
    
     visits_count AS (SELECT
                        sourceid,
                        DATE_TRUNC('day',startts::TIMESTAMP) AS dt_day,
                        COUNT(DISTINCT uid)                        
                FROM
                    visits
                GROUP BY
                    sourceid,
                    dt_day                    
                ORDER BY
                    dt_day ASC)

SELECT
    spend_ad.source_id,
    spend_ad.dt_day,
    SUM(sum / count)
FROM 
    spend_ad
INNER JOIN
    visits_count on spend_ad.dt_day = visits_count.dt_day AND spend_ad.source_id = visits_count.sourceid
GROUP BY
    spend_ad.source_id,
    spend_ad.dt_day
'''
metric = pd.io.sql.read_sql(metric, con=engine)
metric

,source_id,dt_day,sum
0,1,2017-06-01,$10.74
1,1,2017-06-02,$20.75
2,1,2017-06-03,$9.13
3,1,2017-06-04,$9.16
4,1,2017-06-05,$7.13
...,...,...,...
2363,10,2018-05-24,$5.39
2364,10,2018-05-25,$7.40
2365,10,2018-05-28,$7.08
2366,10,2018-05-30,$8.28


#### Python

In [52]:
spend_ad = advertisment_costs.groupby(['source_id','dt'])['costs'].sum().reset_index()
spend_ad.columns = ['source_id','dt','costs']
spend_ad['dt'] = spend_ad['dt'].astype('datetime64[D]')

visits['date_st'] = visits['startts'].astype('datetime64[D]')
visits_count = visits.groupby(['sourceid','date_st'])['uid'].nunique().reset_index()
visits_count.columns = ['source_id','dt','nunique_visits']

total_data = spend_ad.merge(visits_count, on=['dt','source_id'], how='left')
total_data['metric'] = (total_data['costs'] / total_data['nunique_visits']).round(3)
total_data = total_data.groupby(['source_id','dt'])['metric'].sum().reset_index()
total_data

,source_id,dt,metric
0,1,2017-06-01,10.743
1,1,2017-06-02,20.750
2,1,2017-06-03,9.132
3,1,2017-06-04,9.167
4,1,2017-06-05,7.135
...,...,...,...
2537,10,2018-05-27,0.000
2538,10,2018-05-28,7.087
2539,10,2018-05-29,0.000
2540,10,2018-05-30,8.288


#### Вывод
Самый затратный день (с максимальной метрикой 136.595) вышел на 01.08.2017 на 3й рекламный ресурс. Наименьший максимальный показатель (26.74) на рекламу у 9ого рекламного ресурса. На него же тратится наименьшее количество средств.